# Lab 5 - Building an LLM-powered Natural Language to SQL Application

## Introduction

Generative AI and Large Language Models (LLMs) have recently gained a lot of popularity due to their advanced natural language processing and understanding abilities. In 2018, the state of the art NLP task accuracy was set by BERT-large with its 340M parameters and novel transformers architecture. After just a few years, state-of-the-art model size has grown by more than 500x with models such as OpenAI’s 175 billion parameter GPT-3 and the similarly sized open source Falcon 180B by TII, raising the bar on NLP accuracy. The surge in parameter count is caused by the straightforward and empirically proven correlation between model size and performance that bigger often means better. As model repositories like HuggingFace, that provide convenient access to models and NLP tasks such as classification and text/code generation, continue to improve, users are increasingly opting for these large models. Nonetheless, the deployment of these models can present challenges due to their substantial size. 

In our era of data driven enterprises, a lot of data is stored in structured databases. SQL (Structured Query Language) has long been the language of data manipulation, enabling professionals to extract valuable insights from vast databases. However, for many, learning SQL has a steep learning curve and thus remains intimidating. 
With the rise of LLMs, we can now leverage their code generation capabilities to interact with these database using natural language, having SQL only as an intermediate step. 
This allows us to democratise data to non-technical people, build chat-bots on enterprise data, and also assist SQL experts like data scientists with their tasks. 

In this Lab we will explore how to build a Generative AI powered application to query structured data from Amazon Relational Database Service using natural language on AWS.

We will create an RDS Database instance backed by the Amazon Aurora PostgreSQL engine and populate it with synthetic data. Furthermore, we will deploy the [Code Llama](https://ai.meta.com/blog/code-llama-large-language-model-coding/) (7B) model by [Meta](https://ai.meta.com/) using Amazon SageMaker Jumpstart. 
Our database schema of synthetic data looks as follows:

![database schema](./fe/schema.png)

### Background and details

We have two primary types of knowledge for LLMs:

* Parametric knowledge: refers to everything the LLM learned during training and acts as a frozen snapshot of the world for the LLM.
* Source knowledge: covers any information fed into the LLM via the input prompt.

When infusing knowledge into Generative AI powered applications, we need to decide which of these types to target. In previous labs, we have elevated the parametric knowledge through fine tuning. Fine-tuning can be a resource intensive task and many practitioners opt for targeting the source knowledge. Luckily, Meta's Code Llama, comes as a fine-tuned version of Llama-2, for code-secific tasks. This Lab is about using prompt engineering techniques to leverage the code capabilities of our model to query our specific data.  
The application we will be building in a step-by-step process will be a LLM powered dashboard for you to query synthetic data of a made up software company. You will be able to query the database through a frontend and receive information, specific to the underlying data. 


### Workflow

The workflow of our demo application looks as following. We feed the (1) user's question in natural language to our model hosted on an SageMaker Endpoint using some prompt engineering techniques, which we dive deeper into in this Notebook. (2) The LLM converts the question in natural language to a SQL query, which is (3) directly excecuted on our database. Our application is designed in a way to (4) return the results of the query to the user directly, alongside with (5) an answer to the question in Natural Language.

![workflow](./img/workflow.png)

### Architecture
The mentioned architecture is implemented using the three main services: 
* AWS Elastic Beanstalk
* Amazon SageMaker
* Amazon Relational Database Service (RDS)

The architecture diagram shows how these services interact for our application.

![architecture](./img/arch_codellama.png)

### Instructions

#### Prerequisites
To run this Lab, you need 
* a computer with a web browser
* an AWS account with access to `ml.g5.2xlarge` instances
* Access to the Code Llama model by Meta AI through SageMaker Jumpstart

#### Recommended Background 
It will be easier for you to run this workshop if you have:

* Experience with Large Language Models (LLMs) and prompt engineering techniques
* Familiarity with Python or other similar programming languages
* Familiarity with SQL and relational databases
* Experience with Jupyter notebooks
* Beginners level knowledge and experience with SageMaker Hosting/Inference.
* Beginners level knowledge and experience with Large Language Models


#### Target audience
Data Scientists, ML Engineering, ML Infrastructure, MLOps Engineers, Technical Leaders. Intended for customers working with large Generative AI models including Language, Computer vision and Multi-modal use-cases. Customers using Elastic Beanstalk for hosting or experience with SageMaker.

Level of expertise - 400

#### Time to complete
Approximately 1 hour.

## Setup


### Import of required dependencies

For this lab, we will be needing the following packages:

- `sagemaker`: SageMaker SDK for interacting with Amazon SageMaker.
- `boto3`: The boto3 library is the AWS SDK for python

In [ ]:
import sagemaker
import boto3

### Setup the notebook environment

Make sure you are using this notebook in an Amazon SageMaker Notebook Instance for the cell below to work. Here we will retrieve the region specific ARN for our model.

Before we begin with building the application, we need to setup the notebook environment properly. This includes:
* retrieval of the chosen region for later usage
* retrieval of the SageMaker Session object for later usage, to manage interactions with the Amazon SageMaker APIs and any other AWS services needed.


In [ ]:
# Retrieve region
region = boto3.Session().region_name
region

In [ ]:
# Retrieve SageMaker Session
sagemaker_session = sagemaker.Session()

## Deploy the model to an endpoint for real-time inference

[Code Llama](https://ai.meta.com/blog/code-llama-large-language-model-coding/) is a model released by Meta built on top of Llama 2 that is designed to help programmers write high quality, well-documented code more efficiently. It supports major programming languages like Python, C++, Java, and can understand natural language instructions. Code Llama comes in three variants for different applications - a general model, a Python specialized model, and one focused on instruction-following. All variants are available in three sizes - 7B, 13B and 34B parameters - to cover a wide range of use cases. Here we leverage the SQL generation capabilities of the foundational Code Llama 7B model, to strike the right balance between exceptional quality and affordability.


In this section we will be deploying the Code Llama model to a SageMaker endpoint for real time inference. Let us first define an endpoint name.

In [ ]:
endpoint_name = "code-llama-7b"

While creating an endpoint, you can choose the instance type to run the model on. If you don't specify it, it runs on a default instance type, which is an `ml.g5.2xlarge` for Code Llama 7b. We follow this [blog post](https://aws.amazon.com/blogs/machine-learning/code-llama-code-generation-models-from-meta-are-now-available-via-amazon-sagemaker-jumpstart/) for deployment and make use of the default instance.

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).

Keep in mind, if your model is already deployed to an endpoint of the same name, or an endpoint configuration of the same name is already created, then this should fail! Deployment can take a while. For us it took up to 10 minutes to deploy the `Code Llama 7B` model to a `ml.g5.2xlarge` instance.

In [ ]:
from sagemaker.jumpstart.model import JumpStartModel

model = JumpStartModel(model_id="meta-textgeneration-llama-codellama-7b")
predictor = model.deploy(
    endpoint_name=endpoint_name
    # instance_type="ml.g5.2xlarge", # uncomment to use another instance
    )

Your model should now be deployed. Let us now try interacting with the model. 

## Interact with the model (prompting) 

While interacting with our model, there is some things we must think about. First of all the model has arguments, also known as parameters, which let you control the models responses.
The main thing we need to think about here however, is the input, that we send into out model, which is called the prompt. Think of the model as a sentence completing algorithm, where it simply completes our prompt.
First we try the [zero shot prompting](https://www.promptingguide.ai/techniques/zeroshot) method. 

#### Zero-Shot prompting

We ask the model to create an SQL statement given our instruction in natural language as you can see below. This should generate us a correct SQL statement and your output should look similar to this: 
```SQL 
SELECT * FROM employees;
``` 
Next to our prompt, we also need to feed in hyperparameters. The hyperparameters are: 
* `max_new_tokens`, define the maximum number of tokens to generate in this call.
* `temperature`, which let's you control how deterministic a model's responses are. Setting temperature to 1.0 samples directly from the model distribution. Lower (higher) values increase the chance of sampling higher (lower) probability tokens. A value of 0 essentially disables sampling and results in greedy decoding, where the most likely token is chosen at every step. To find out more about temperature you can have a look [here](https://www.promptingguide.ai/introduction/settings.en). We choose relatively low temperature, as we want deterministic SQL results.
* `top_p` is also a sampling technique, with which you can control how deterministic the models generation is. A lower value keeps the answers factual, while higher values make the responses more diverse. Intuitively, instead of sampling only from the most likely _k_ words, this method samples from the smallest possible set of words, whose summed up probability exeeds `p`. This is also known as nucleus sampling.

Furthermore, you must explicitly accept the EULA to deploy the model via SageMaker SDK. Note that by default, `accept_eula` is set to `false`. You need to set `accept_eula=true` to invoke the endpoint successfully. By doing so, you accept the user license agreement and acceptable use policy. You can also [download](https://ai.meta.com/resources/models-and-libraries/llama-downloads/) the license agreement.



In [ ]:
instruction = """
Create an executable SQL statement from instruction:

Instruction:
Who are the employees of the company?

SQL Query:
"""

payload = {
   "inputs": instruction,
   "parameters": {"max_new_tokens": 10, "temperature": 0.2, "top_p": 0.9}
}

response = predictor.predict(payload, custom_attributes="accept_eula=true")
print(response["generated_text"])

While the zero-shot method works well for simple queries, it would fail for more complex ones. Here we ask it to list all the SDEs of the company. SDE is an [ambigous](https://en.wikipedia.org/wiki/SDE) term used for Software Engineers or to be precise Software Development Engineers. 

 We expect our model to understand, that by `SDEs`, we mean employees, who have the designation `Software Engineer` but without context, the model has too little information to create a correct SQL query and might interpret the term SDE as something else. Also, due to lack of context, it might make up table and column names for the database. For some context, this is how our `employees` table looks like:

| employee_id	|   first_name  |	last_name	|   designation             |   project_id  |   email                   |   manager_id  |
| ----------- | ----------- | ----------- | ----------- | ----------- | ----------- |----------- |
| 91021111      |	Max	        |   Mustermann	|   Software Engineer	    |   283921	    |   mustermann@company.com  |	48234012|
| 91021151      |	Ned	        |   Stark       |   Chief Technology Officer|	- |   nstark@company.com	    |   -   |
| 10921212      |	Dolly       |	Meinhard	|   Project Manager         |	131032      |	dmein@company.com       |	91021111|
| 48234012      |	Josey   	|   Rojas	    |   Product Manager        	|   283921	    |   jroj@company.com        |	91021111|
| 43204121      |	Fidel       |   Wind	    |   Software Engineer	    |   311092      |	fwind@company.com	    |   1823012 |
| 1823012   	|   Peter       |	Kabel	    |   Software Engineer	    |   131032      |	pkabel@company.com      |	13220812|
| 13220812      |	Velma       |	Cabello     |	Data Scientist          |	933012      |	vcabello@company.com    |	91021111|


In [ ]:
instruction = """
Create an executable SQL statement from instruction:

Instruction:
Who are the SDEs?

SQL Query:
"""

payload = {
   "inputs": instruction,
   "parameters": {"max_new_tokens": 15, "temperature": 0.2, "top_p": 0.9}
}

response = predictor.predict(payload, custom_attributes="accept_eula=true")
print(response["generated_text"])

To tackle problems, which we mentioned earlier, lets provide more context to our model and try a [few-shot](https://www.promptingguide.ai/techniques/fewshot) approach. We will split this into multiple steps:
1. curate our prompt template using LangChains [`PromptTemplate`](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/) module
2. feed in our database context 
3. add in some example queries for our few-shot approach. 

[LangChain](https://www.langchain.com/) is a framework built to simplify the creation of applications using Large Language Models (LLMs). For our application we use it throughout our prompting process. We leverage its integration with SageMaker Endpoints and Relational Databases to connect directly with our services. For more information on what it offers, take a look at the [documentation](https://python.langchain.com/docs/get_started/introduction).

In [ ]:
from langchain import PromptTemplate

TEMPLATE = """Given an input question, create a syntactically correct {dialect} query to run.
Use the following format:

Question: "Question here"
SQLQuery:
"SQL Query to run"

Only use the following tables:

{table_info}.

Some examples of SQL queries that correspond to questions are:

{few_shot_examples}

Question: {input}
SQLQuery:
"""

CUSTOM_PROMPT = PromptTemplate(
    input_variables=["input", "few_shot_examples", "table_info", "dialect"], template=TEMPLATE
)

#### Table Context
Now we will add some information about our database tables into the prompt. For this we add information from all tables following the best practices specified in [Rajkumar et. al., 2022](https://arxiv.org/abs/2204.00498), in which they found the `CREATE TABLE` table definition syntax to give promising results. Additionally we add in the top 4 rows of each table, in the same way as done in the paper.

Here we define our table information `table_info` as a variable and pass it as an argument into our prompt in a later cell.
<a id='db_context_cell'></a>


In [ ]:
# define table info

table_info = """

CREATE TABLE employees (
	employee_id INTEGER NOT NULL, 
	first_name VARCHAR(50) NOT NULL, 
	last_name VARCHAR(50) NOT NULL, 
	designation VARCHAR(50) NOT NULL, 
	project_id INTEGER, 
	email VARCHAR(255) NOT NULL, 
	manager_id INTEGER, 
	CONSTRAINT employees_pkey PRIMARY KEY (employee_id), 
	CONSTRAINT employees_manager_id_fkey FOREIGN KEY(manager_id) REFERENCES employees (employee_id), 
	CONSTRAINT employees_project_id_fkey FOREIGN KEY(project_id) REFERENCES projects (project_id), 
	CONSTRAINT employees_email_key UNIQUE (email)
)

/*
4 rows from employees table:
employee_id	first_name	last_name	designation	project_id	email	manager_id
91021111	Max	Mustermann	Software Engineer	283921	mustermann@company.com	48234012
91021151	Ned	Stark	Chief Technology Officer	None	nstark@company.com	None
10921212	Dolly	Meinhard	Project Manager	131032	dmein@company.com	91021111
48234012	Josey	Rojas	Product Manager	283921	jroj@company.com	91021111
*/


CREATE TABLE projects (
	project_id INTEGER NOT NULL, 
	project_name VARCHAR(255) NOT NULL, 
	cutomer VARCHAR(25), 
	CONSTRAINT projects_pkey PRIMARY KEY (project_id)
)

/*
4 rows from projects table:
project_id	project_name	cutomer
283921	Restaurant Management App	The Mozzarella Fellas
131032	Garden Planner	Flamingo Gardens
933012	Music generator	ElvisAI
311092	Weather forecasting system	Flamingo Gardens
*/


CREATE TABLE timelog (
	entry_id INTEGER NOT NULL, 
	employee INTEGER, 
	working_day DATE, 
	entered_hours INTEGER, 
	CONSTRAINT timelog_pkey PRIMARY KEY (entry_id), 
	CONSTRAINT timelog_employee_fkey FOREIGN KEY(employee) REFERENCES employees (employee_id)
)

/*
4 rows from timelog table:
entry_id	employee	working_day	entered_hours
10000002	91021111	2022-01-03	3
10000003	91021151	2022-01-03	4
10000004	10921212	2022-01-03	7
10000005	48234012	2022-01-03	4
*/

"""

#### Few-Shot prompting
Now we define some examples for our model, which we will use in our prompt as well. Here we show, in which style and format we want the answer through examples. We try to cover as many different query types as possible to let the model generalize as well as possible. We limit ourselves to 6 samples, because we don't want to bias our model towards the sample queries when running inference. The number of examples that should be chosen is heaviliy dependent on the size and complexity of the data and should be chosen on a case to case basis. For more complex databases it might make sense to store multiple few-shot examples using a vector database and use only a few of them dynamically for each query. More information and an example on this approach can be found [here](https://python.langchain.com/docs/use_cases/qa_structured/sql#including-dynamic-few-shot-examples) and for more information on Retrieval Augmented Generation (RAG) see [here](https://docs.aws.amazon.com/sagemaker/latest/dg/jumpstart-foundation-models-customize-rag.html)


In [ ]:
# define few shot examples
few_shot_examples = """

Question: Who worked the most hours in 2022?
SQL Query:
SELECT e.first_name, e.last_name, SUM(t.entered_hours) AS total_hours_worked
FROM employees e
JOIN timelog t ON e.employee_id = t.employee_id
WHERE EXTRACT(YEAR FROM t.working_day) = 2022
GROUP BY e.employee_id, e.first_name, e.last_name
ORDER BY total_hours_worked DESC
LIMIT 1;

##

Question: How many Software Engineers does the company have?
SQL Query:
SELECT COUNT(*) from employees
WHERE designation='Software Engineer';

##

Question: How many hours did Velma work in July 2022?
SQL Query:
SELECT SUM(t.entered_hours) AS total_hours_worked
FROM employees e
JOIN timelog t ON e.employee_id = t.employee
WHERE e.first_name = 'Velma'
  AND EXTRACT(YEAR FROM t.working_day) = 2022
  AND EXTRACT(MONTH FROM t.working_day) = 7;

##

Question: Who is working on the Music generator project?
SQL Query:
SELECT * FROM employees
WHERE project_id=(
SELECT project_id FROM projects
WHERE project_name = 'Music generator'
);

##

Question: Who works under Max?
SQL Query:
SELECT * FROM employees
WHERE manager_id=(
SELECT employee_id FROM employees
WHERE first_name = 'Max');

##

Question: Who worked the least hours in April 2022?
SQL Query:
SELECT e.first_name, e.last_name, SUM(t.entered_hours) AS total_hours_worked
FROM employees e
JOIN timelog t ON e.employee_id = t.employee
WHERE EXTRACT(YEAR FROM t.working_day) = 2022
  AND EXTRACT(MONTH FROM t.working_day) = 4
GROUP BY e.employee_id, e.first_name, e.last_name
ORDER BY total_hours_worked
LIMIT 1;

##

"""



Let's try it out with the same prompt as before. After running this cell, we can see that these results match [our database schema](#schema) more and could be executed on the database directly. 
As a reminder, our prompt from earlier looked like this:
```python
TEMPLATE = """Given an input question, create a syntactically correct {dialect} query to run.
Use the following format:

Question: "Question here"
SQLQuery:
"SQL Query to run"

Only use the following tables:

{table_info}.

Some examples of SQL queries that correspond to questions are:

{few_shot_examples}

Question: {input}
SQLQuery:
"""

CUSTOM_PROMPT = PromptTemplate(
    input_variables=["input", "few_shot_examples", "table_info", "dialect"], template=TEMPLATE
)
```

In [ ]:
input="Who are the SDEs?"

instruction = CUSTOM_PROMPT.format(
    input=input,
    table_info=table_info,
    dialect="PostgreSQL",
    few_shot_examples=few_shot_examples
)

payload = {
   "inputs": instruction,
   "parameters": {"max_new_tokens": 20, "temperature": 0.2, "top_p": 0.9}
}

response = predictor.predict(payload, custom_attributes="accept_eula=true")
print(response["generated_text"])

We've now seen and validated that this works, as a next step we want to make this more scalable and manageable. Let's try to automate this.

## Connect to the database with Langchain

Now that we have our model deployed, let us automate the prompting process, eliminating the necessity of hardcoding the table information. For this let's create a database. To follow best practice and simplicity, we will create a database locally next to this notebook to try out the prompting process instead of connecting to an RDS instance. Later, when we create the frontend and deploy the architecture, we will interact with our database cluster on RDS directly. Let's create a database locally and populate it with our data.

In [ ]:
!chmod +x setup_db_debian.sh && ./setup_db_debian.sh

Let's check if the connection works properly, by executing an SQL statement.

In [ ]:
!psql -d "companydb" -c "SELECT * FROM employees;"

Now connect to the database using LangChains [SQLDatabase]() module. Here we have the option to specify which tables to include and how many sample rows we want in our table information. Now running this cell should leave you with exacly the same table info as [before](#3-interact-with-the-model-prompting).

In [ ]:
from langchain import SQLDatabase

RDS_PORT = 5432
RDS_DB_NAME = "companydb" 
RDS_URI = f"postgresql+psycopg2://:@:{RDS_PORT}/{RDS_DB_NAME}"
   
db = SQLDatabase.from_uri(RDS_URI,
                           include_tables=["employees", "projects", "timelog"],
                           sample_rows_in_table_info=4)
print(db.table_info)

Now let us check if the connection to our database through the LangChain module works. 

In [ ]:
# check if the connection works.
db.run("SELECT * from employees")

## Automate the prompting process using Langchain

Now that we have a database connection in place using Langchain, we will automate the prompting process and avoid having to hardcode the table info like we did before, but rather fetch it dynamically. First we define a ContentHandler class to encode and decode the communication with the model, as it has been shown [here](https://python.langchain.com/docs/integrations/llms/sagemaker).

In [ ]:
import json 
from langchain.llms.sagemaker_endpoint import LLMContentHandler, SagemakerEndpoint
from langchain.chains import create_sql_query_chain

class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
        input_str = json.dumps(
            {"inputs" : prompt,
            "parameters" : {**model_kwargs}})
        return input_str.encode('utf-8')
    
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json["generated_text"]


Now we wrap the communication with the model with LangChain as well and recieve it as a python object using LangChains `SagemakerEndpoint` class. This will make the communication with our SageMaker endpoint easier. As arguments we have to feed it our endpoint_name, region, the content handler, which we just defined and the aformentioned hyperparameters. 

In [ ]:
# Wrap the communication with the model with Langchain as well
content_handler = ContentHandler()

parameters = {"max_new_tokens": 100, "temperature": 0.2, "top_p": 0.9}

llm_codellama = SagemakerEndpoint(
    endpoint_name=endpoint_name,
    region_name=region,
    model_kwargs=parameters,
    endpoint_kwargs={"CustomAttributes": 'accept_eula=true'},
    content_handler=content_handler
)

Finally, let's run our query end to end using LangChain's `create_sql_query_chain` and format our answer the way we need it. For more information on `create_sql_query_chain` you can have a look at the [documentation](https://api.python.langchain.com/en/latest/chains/langchain.chains.sql_database.query.create_sql_query_chain.html).

In [ ]:
input = "Who are the SDEs?"
prompt = CUSTOM_PROMPT.format(
    input=input,
    table_info=db.table_info,
    dialect="PostgreSQL",
    few_shot_examples=few_shot_examples
)

chain = create_sql_query_chain(llm_codellama, db) 
result = chain.invoke({"question": prompt})

# We cut off everything after our SQL statement and add a semicolon if it wasn't already there.
result = result.split(";")[0] + ";"

print(result)

We can run this query now directly against our database. 

In [ ]:
# run this code directly on the db
import os
os.system(f'psql -d companydb -c "{result}"')

## Create Frontend and deploy architecture

Now that we have our model deployed on an endpoint for real-time inference, and learned how to query a Postgresql Database using Natural Language, let's deploy the rest of the architecture. For this we use AWS CloudFormation, which lets you model, provision, and manage AWS and third-party resources by treating infrastructure as code. For more information on Cloudformation and its features, have a look [here](https://aws.amazon.com/cloudformation/).

<span style="color:red"> *⚠️Warning: We give Elastic Beanstalk full SageMaker access, which is over permissive and should not be used in production*</span>

In [ ]:
import os
# define bucket and zip file
sess = sagemaker.Session()
bucket = sess.default_bucket() # Set default S3 bucket
zip_file = "myapp.zip"
rds_master_username = "dbadmin"
rds_master_password = "12345678" # change password here

# deploy the architecture
!chmod +x deploy_cf.sh
os.system(f"./deploy_cf.sh {bucket} {zip_file} {rds_master_username} {rds_master_password}")

Let us wait for the Cloudformation stack to finish deploying, before we continue interacting with it. This should take about 5 minutes

In [ ]:
!aws cloudformation wait stack-create-complete --stack-name text2sql-app

Now we add our database credentials to AWS Systems Manager Parameter Store, so our application can access our database securely. 

In [ ]:
rds_client = boto3.client("rds")
for cluster in rds_client.describe_db_clusters()['DBClusters']:
    if 'DatabaseName' in cluster.keys() and cluster['DatabaseName'] == 'CompanyDatabase':
        rds_endpoint = cluster['Endpoint'] # get RDS endpoint

assert rds_endpoint is not None

# Upload Database credentials to Parameterstore
ssm_client = boto3.client("ssm")
ssm_client.put_parameter(
    Name="text2sql_db_endpoint",
    Type="SecureString",
    Value=rds_endpoint,
    Overwrite=True
)
ssm_client.put_parameter(
    Name="text2sql_db_user",
    Type="SecureString",
    Value=rds_master_username,
    Overwrite=True

)
ssm_client.put_parameter(
    Name="text2sql_db_password",
    Type="SecureString",
    Value=rds_master_password,
    Overwrite=True
)

# Print the IP address of the deployed demo!
cf_client = boto3.client('cloudformation')
stackname = 'text2sql-app'

response = cf_client.describe_stacks(StackName=stackname)
outputs = response["Stacks"][0]["Outputs"]
for output in outputs:
    keyName = output["OutputKey"]
    if keyName == "StreamlitURL":
        print(f"Paste the following IP address in your browser to view your application: {output['OutputValue']} 🤖")

**Congratulations! You have successfully built and deployed a Natural Language to SQL application on AWS! 🎉**

## Teardown

To avoid being charged, let us delete the resources we created throughout this notebook. This includes the SageMaker Endpoint, SageMaker Endpoint Configuration, Parameters and the model itself.

### Delete the endpoint and model

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()

### Delete the Cloudformation stack

In [ ]:
!aws cloudformation delete-stack --stack-name text2sql-app

### Delete Parameters

In [ ]:
!aws ssm delete-parameters --names "text2sql_db_endpoint" "text2sql_db_user" "text2sql_db_password"

## Conclusion

In this notebook, we saw a simple workflow on how to build a Natural Language to SQL demo leveraging the code generation capabilities of LLMs using Amazon SageMaker Jumpstart. Later we deployed the entire containerized application through AWS Elastic Beanstalk and built a frontend around it using the open source library Streamlit. 
We have the following learnings:
* We deployed the Code Llama 7B model using SageMaker Jumpstart
* We learned to interact with our model using the SageMaker SDK
* We used LangChains `PromptTemplate` module to define our prompt template, which includes table information and few shot examples.
* We explored LangChains `create_sql_query_chain` module, to generate an SQL query
* We connected all these components including the execution of the query on our database.

You can adapt this notebook and also the application to your data by connecting it to your database and using few-shot examples specific to your data. This notebook contains static few-shot examples. For a larger corpus of data, it might make sense to consider [including dynamic few-shot examples](https://python.langchain.com/docs/use_cases/qa_structured/sql#including-dynamic-few-shot-examples). Furthermore, the table information can also be dynamic for every question, eliminating the need of including all the tables. For this, you can take a look at [Retrieval Augmented Generation (RAG)](https://docs.aws.amazon.com/sagemaker/latest/dg/jumpstart-foundation-models-customize-rag.html)

While this notebook contains a simple workflow on how to build a Natural Language to SQL demo, for a deeper dive, please refer to [this](https://arxiv.org/pdf/2308.15363.pdf) benchmark evaluation on this use case, including Supervised Fine Tuning for Text-to-SQL. If you want to explore how to build applications around our use case and operationalize everything, see [here](https://aws.amazon.com/blogs/machine-learning/fmops-llmops-operationalize-generative-ai-and-differences-with-mlops/).
